In [2]:
import psycopg2

In [3]:
conn_string = "host='localhost' dbname ='dvdrental' user='postgres' password='wns123'"
conn = psycopg2.connect(conn_string)
cur = conn.cursor()

In [4]:
cur.execute(
    "SELECT * FROM payment WHERE amount <= 1 OR amount >= 8 ORDER BY payment_date OFFSET 10 ROWS FETCH FIRST 10 ROWS ONLY"
    )
result = cur.fetchall()
result[:2]

[(17857,
  434,
  1,
  1225,
  Decimal('0.99'),
  datetime.datetime(2007, 2, 15, 2, 14, 1, 996577)),
 (17901,
  447,
  2,
  1230,
  Decimal('0.99'),
  datetime.datetime(2007, 2, 15, 2, 32, 35, 996577))]

In [5]:
cur.execute(
    "SELECT customer_id, rental_id, return_date \
    FROM rental \
    WHERE customer_id IN (1,2) \
    ORDER BY return_date DESC;"
    )
result = cur.fetchall()
result[:2]

[(2, 15145, datetime.datetime(2005, 8, 31, 15, 51, 4)),
 (1, 15315, datetime.datetime(2005, 8, 30, 1, 51, 46))]

In [6]:
cur.execute(
    "SELECT first_name, last_name FROM customer \
    WHERE customer_id IN \
    (SELECT customer_id \
    FROM rental \
    WHERE CAST (return_date AS date) = '2005-05-27');"
    )
result = cur.fetchall()
result[:2]

[('Pamela', 'Baker'), ('Frances', 'Parker')]

In [7]:
cur.execute(
    "SELECT payment_id, payment_date, to_char(payment_date, 'yyyy-mm-dd-hh') AS date, cast(payment_date AS date) \
    FROM payment \
    WHERE to_char(payment_date, 'yyyy-mm-dd') BETWEEN '2007-02-07' AND '2007-02-15';"
    )
result = cur.fetchall()
result[:2]

[(17503,
  datetime.datetime(2007, 2, 15, 22, 25, 46, 996577),
  '2007-02-15-10',
  datetime.date(2007, 2, 15)),
 (17519,
  datetime.datetime(2007, 2, 15, 10, 54, 44, 996577),
  '2007-02-15-10',
  datetime.date(2007, 2, 15))]

In [8]:
cur.execute(
    "SELECT payment_id, payment_date, to_char(payment_date, 'yyyy-mm-dd-hh') AS date, cast(payment_date AS date) \
    FROM payment \
    WHERE to_char(payment_date, 'yyyy-mm-dd') BETWEEN '2007-02-07' AND '2007-02-15';"
    )
result = cur.fetchall()
result[:2]

[(17503,
  datetime.datetime(2007, 2, 15, 22, 25, 46, 996577),
  '2007-02-15-10',
  datetime.date(2007, 2, 15)),
 (17519,
  datetime.datetime(2007, 2, 15, 10, 54, 44, 996577),
  '2007-02-15-10',
  datetime.date(2007, 2, 15))]

In [10]:
cur.execute(
    "SELECT e.first_name || ' ' ||e.last_name employee,\
    m.first_name || ' ' ||m.last_name manager \
    FROM employee e \
    LEFT  JOIN employee m \
    ON m.employee_id = e.manager_id \
    ORDER BY manager"
    )
result = cur.fetchall()
result[:2]

[('Anna Reeves', 'Ava Christensen'), ('Sau Norman', 'Ava Christensen')]

In [14]:
cur.execute(
    "SELECT sum(amount) AS total, b.customer_id, b.first_name, b.last_name \
    FROM payment a \
    LEFT JOIN customer b \
    ON a.customer_id = b.customer_id \
    GROUP BY b.customer_id \
    HAVING sum(amount) > 160 \
    ORDER BY total DESC \
    LIMIT 2 OFFSET 0"
    )
result = cur.fetchall()
result[:2]

InterfaceError: cursor already closed

In [11]:
col_name = [desc[0] for desc in cur.description]
df = pd.DataFrame(result, columns=col_name)

In [12]:
df.tail()

,employee,manager
3,Kelsie Hays,Hassan Conner
4,Tory Goff,Hassan Conner
5,Hassan Conner,Windy Hays
6,Ava Christensen,Windy Hays
7,Windy Hays,None


In [13]:
cur.close()
conn.close()